In [1]:
import numpy as np
from paho.mqtt.client import Client
import time
import threading
import socket
from tkinter import *

In [2]:
class Reciever_MQTT():
    def on_connect ( clinet, userdata, flags, rc):
        self.subscribe("flowertest")
    def on_message(self, client, userdata, msg):
        global door
        if (msg.topic == 'flowertest'):
            door.change(msg.payload)
    def __init__(self):
        self.client = Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.client.username_pw_set("njelhmbg","FLLRBnPB4UiD")
        self.client.connect("hairdresser.cloudmqtt.com", 16008,60)
    def listen(self):
        self.client.loop_forever()
        

In [3]:
class Reciever_SOCKET:
    def __init__(self,ip, port):
        self.ip = ip
        self.port = port
    def listen(self):
        global door
        with socket.socket() as sock:
            sock.bind((self.ip, self.port))
            sock.listen()
            while True:
                conn, addr = sock.accept()
                conn.settimeout(5)
                with conn:
                    try:
                        data = conn.recv(1024)
                    except socket.timeout:
                        print('close connection by timeout')
                        break
                    if not data:
                        break
                    door.change(data.decode('utf8'))
    def stop(self):
        self.sock.stop()
                    

In [4]:
class DoorState(object):
    name = "state"
    allowed = []
    def __str__(self):
        """This is 'magic method', which changes the behavior of instance of class, when it is printed """ 
        return self.name

    def switch(self, state):
        """ Switch to new state """
        if state.name in self.allowed:
            print ('Current:',self,' => switched to new state',state.name)
            self.__class__ = state
        else:
            print ('Current:',self,' => switching to',state.name,'not possible.')

class Disabled(DoorState):
    name = "disabled"
    allowed = ['error', 'enabled']

class Enabled(DoorState):
    name = "enabled"
    allowed = ['error', 'disabled','closing','opens']

class Closing(DoorState):
    """ State of being in suspended mode after switched on """
    name = "closing"
    allowed = ['error', 'closed', 'opens']

class Opens(DoorState):
    """ State of being in hibernation after powered on """
    name = "opens"
    allowed = ['error', 'closing', 'open']
class Closed(DoorState):
    name = "closed"
    allowed = ['error', 'opens', 'disabled']
class Open(DoorState):
    name = "open"
    allowed = ['error', 'closing', 'disabled']
class Error(DoorState):
    name = "error"
    allowed = ['enabled', 'disabled']

In [5]:
class Door:
    state_dict = {
        'OFF': Disabled,
        'ON': Enabled,
        'CLOSE': Closing,
        'OPEN': Opens,
        'ERROR': Error,
    }
    def __init__(self):
        self.state = Disabled()
        self.open_time = self.close_time = 5 
    def change(self, message):
        """ Change state """
        try:
            if(self.state_dict[message] != self.state.name):
                if(self.state_dict[message].name == 'opens'):
                    self.state.switch(Opens)
                    time.sleep(self.open_time)
                    self.state.switch(Open)
                elif(self.state_dict[message].name == 'closing'):
                    self.state.switch(Closing)
                    time.sleep(self.close_time)
                    self.state.switch(Closed)
                else:
                    self.state.switch(self.state_dict[message])
            else:
                pass            
        except KeyError:
            print('###Error###')
            self.state.switch(self.state_dict['ERROR'])
    def get_state():
        return self.state.name

In [ ]:
if __name__ == '__main__':
    ip = '127.0.0.1'
    port = 5001
    reciever = Reciever_SOCKET(ip, port)
    door = Door()
    reciever.listen()
    